## Perspective Aware Sentiment Analysis

##### Comparing Traditional vs LLM Models

In [ ]:
import feedparser
import urllib.parse
from dateutil import parser
from dotenv import load_dotenv

def fetch_google_news(query: str, hours: int = 24, limit: int = 50):
    """
    Fetch recent Google News results in a list-of-dicts format.

    Args:
        query (str): Search term, e.g., "HDFC Bank"
        hours (int): Time window in hours (e.g., 24)
        limit (int): Max number of news articles

    Returns:
        list[dict]: [{'title': ..., 'link': ..., 'date': ...}, ...]
    """
    encoded_query = urllib.parse.quote(query)
    feed_url = (
        f"https://news.google.com/rss/search?"
        f"q={encoded_query}+when:{hours}h&hl=en-IN&gl=IN&ceid=IN:en"
    )

    feed = feedparser.parse(feed_url)
    articles = []

    for entry in feed.entries[:limit]:
        title = entry.title
        link = entry.link
        date = parser.parse(entry.published).isoformat() if "published" in entry else None

        articles.append({
            "title": title,
            "link": link,
            "date": date
        })

    return articles


In [ ]:
news = fetch_google_news("election", hours=12, limit=10)
for item in news:
    print(item)


{'title': 'Kerala Local Body Elections Scheduled in Two Phases on 9 and 11 December 2025 - Newsonair', 'link': 'https://news.google.com/rss/articles/CBMiqwFBVV95cUxNMkJMNkxtT2xzSkxxVmdrcDh1OEQ3OHlaVFpTTHR2MEhxaS1RZjZBMmhfWjhlVjZnZllBUlBFUHIzd01sbi1fanhVSFFNd1JYWFk4RHFxakFjTF9OOWpSRUtlRnUxUmprMWNocWdZRTYtRzhMLThsMVpkT2tjdU5kSGZ2bk92RmVIZmNKdXR6TjVaRUhtRWQ0U3FwUkdjY0NqLWQ4LWRla1dTQ2M?oc=5', 'date': '2025-11-13T08:31:17+00:00'}
{'title': 'Bihar Election 2025 LIVE Updates: Bihar readies for counting of votes to decide whether state would see N... - The Economic Times', 'link': 'https://news.google.com/rss/articles/CBMi5wJBVV95cUxNNmstajVGWk04eFZ3NHBxTEViT1o4WHJMVHNqbFZlV0xfeGZ4VTNEcXpMeHFZeE1pSWJPa0EtMDZEWTRIMFN5MHprU21yMGk3bEpuRURESEsxSzZVMXBqMDB0d0dCWk5VWFF4bmNRMkcyNnBDRE9OM1llSFRJZjV3eVQ4VDdPMUlpX0R2U3RrM2VWZjM4aUdZek8yeFptaEhaNFB3NVVDa0QzYndrdWZTMmtHa3RJRVVNdEpiaG5UNUE1SWtNdllTQjA2MnlpeDhhcHlQSFpfSkpmR0xwa3pqekIwMXpxTFFDT2s0ZF9ja04xMDQ5aWZyblplZUNzb1F4SzlHTkNDNlVUdlRYcDRneTVETV9BdjRlSk

In [ ]:
titles = [item["title"] for item in news]
print(titles)

['Kerala Local Body Elections Scheduled in Two Phases on 9 and 11 December 2025 - Newsonair', 'Bihar Election 2025 LIVE Updates: Bihar readies for counting of votes to decide whether state would see N... - The Economic Times', 'Bihar Election 2025 Live Updates: Tejashwi speaks to RJD candidates, says ready to ‘deal with unconstitutional activity’ | India News - Hindustan Times', 'Bihar election: Can Modi buck Gen Z rage in India’s youngest state? - Al Jazeera', 'Bihar Assembly Election 2025: where and how to watch the results live - The Hindu', 'Bihar election result 2025: Date, time and where to watch live counting - The Times of India', 'Bihar Assembly Election Results 2025 Live Updates: Day before counting of votes, NDA and MGB camps confident of ‘clear victory’ - The Indian Express', 'Bihar Election Result 2025: Key Constituency to watch - India Today', 'ITTF Executive Board Election to Take Place Online on Saturday 15 November - International Table Tennis Federation', 'Bihar Elect

In [ ]:
for i in titles :
  print(i)

Kerala Local Body Elections Scheduled in Two Phases on 9 and 11 December 2025 - Newsonair
Bihar Election 2025 LIVE Updates: Bihar readies for counting of votes to decide whether state would see N... - The Economic Times
Bihar Election 2025 Live Updates: Tejashwi speaks to RJD candidates, says ready to ‘deal with unconstitutional activity’ | India News - Hindustan Times
Bihar election: Can Modi buck Gen Z rage in India’s youngest state? - Al Jazeera
Bihar Assembly Election 2025: where and how to watch the results live - The Hindu
Bihar election result 2025: Date, time and where to watch live counting - The Times of India
Bihar Assembly Election Results 2025 Live Updates: Day before counting of votes, NDA and MGB camps confident of ‘clear victory’ - The Indian Express
Bihar Election Result 2025: Key Constituency to watch - India Today
ITTF Executive Board Election to Take Place Online on Saturday 15 November - International Table Tennis Federation
Bihar Election Results 2025: Date, Time 

In [ ]:
data = [
'Kerala Local Body Elections Scheduled in Two Phases on 9 and 11 December 2025 - Newsonair',
'Bihar Election 2025 LIVE Updates: Bihar readies for counting of votes to decide whether state would see N... - The Economic Times',
'Bihar Election 2025 Live Updates: Tejashwi speaks to RJD candidates, says ready to ‘deal with unconstitutional activity’ | India News - Hindustan Times',
'Bihar election: Can Modi buck Gen Z rage in India’s youngest state? - Al Jazeera',
'Bihar Assembly Election 2025: where and how to watch the results live - The Hindu',
'Bihar election result 2025: Date, time and where to watch live counting - The Times of India',
'Bihar Assembly Election Results 2025 Live Updates: Day before counting of votes, NDA and MGB camps confident of ‘clear victory’ - The Indian Express',
'Bihar Election Result 2025: Key Constituency to watch - India Today',
'ITTF Executive Board Election to Take Place Online on Saturday 15 November - International Table Tennis Federation',
'Bihar Election Results 2025: Date, Time And Where To Watch Live Vote Counting Updates - News18'
]

In [ ]:
from transformers import pipeline

# Load a simple general-purpose sentiment model
sentiment_model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

def analyze_sentiments(news_items):
    """
    Given a list of news dictionaries [{'title': ..., 'link': ..., 'date': ...}],
    return the same list with a 'sentiment' key added.
    """
    results = []
    for item in news_items:
        sentiment = sentiment_model(item)[0]  # e.g. {'label': 'POSITIVE', 'score': 0.997}
        results.append({
            "title": item,
            "sentiment": sentiment["label"],
            "confidence": round(sentiment["score"], 3)
        })
    return results


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [ ]:
data = ["Milwaukee Bucks star Giannis Antetokounmpo sustained a left groin strain during a loss to the Cleveland Cavaliers. His absence affected the team's performance significantly. Donovan Mitchell led Cleveland with 37 points."]
sentiment_results = analyze_sentiments(data)

for item in sentiment_results:
    print(f"{item['sentiment']} ({item['confidence']}): {item['title']}")


neutral (0.593): Milwaukee Bucks star Giannis Antetokounmpo sustained a left groin strain during a loss to the Cleveland Cavaliers. His absence affected the team's performance significantly. Donovan Mitchell led Cleveland with 37 points.


In [ ]:
from openai import OpenAI

# Initialize OpenAI client


load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def perspective_sentiment(titles, perspective):
    """
    Uses GPT to analyze sentiment of news headlines from a specific perspective.

    Args:
        titles (list[str]): List of news headlines.
        perspective (str): Description of the user's viewpoint
                           (e.g. "I support BJP" or "I am neutral").

    Returns:
        list[dict]: [{'title': ..., 'sentiment': ..., 'reason': ...}, ...]
    """

    results = []

    system_prompt = f"""
    You are a sentiment analysis assistant.
    For each headline, give sentiment from the perspective of this person: "{perspective}".
    Reply ONLY in JSON list format like:
    [
      {{"title": "headline 1", "sentiment": "Positive", "score": "range from 0 to 1"}},
      ...
    ]
    Possible sentiments: Positive, Negative, Neutral.
    Keep explanations short (max 1 line).
    """

    # Join all titles into a single string to send as input
    user_prompt = "\n".join([f"- {t}" for t in titles])

    response = client.chat.completions.create(
        model="gpt-4o-mini",  # or gpt-4o for better reasoning
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3,
    )

    try:
        # Parse GPT response as JSON
        import json
        results = json.loads(response.choices[0].message.content)
    except Exception:
        # Fallback if GPT output isn't valid JSON
        results = [{"title": t, "sentiment": "Error", "reason": "Parsing failed"} for t in titles]

    return results


In [ ]:
perspective = "I am a fan of Giannis"

results = perspective_sentiment(data, perspective)

for r in results:
    print(f"{r['sentiment']}: {r['title']} — {r['score']}")


Negative: Milwaukee Bucks star Giannis Antetokounmpo sustained a left groin strain during a loss to the Cleveland Cavaliers. His absence affected the team's performance significantly. Donovan Mitchell led Cleveland with 37 points. — 0.12


In [ ]:
perspective = "I don't support BJP"

results = perspective_sentiment(data, perspective)

for r in results:
    print(f"{r['sentiment']}: {r['title']} — {r['reason']}")

Neutral: Kerala Local Body Elections Scheduled in Two Phases on 9 and 11 December 2025 - Newsonair — The headline is informative without any political bias.
Neutral: Bihar Election 2025 LIVE Updates: Bihar readies for counting of votes to decide whether state would see N... - The Economic Times — Focuses on the election process without expressing support for any party.
Positive: Bihar Election 2025 Live Updates: Tejashwi speaks to RJD candidates, says ready to ‘deal with unconstitutional activity’ | India News - Hindustan Times — Supports opposition against unconstitutional activities, aligning with anti-BJP sentiment.
Negative: Bihar election: Can Modi buck Gen Z rage in India’s youngest state? - Al Jazeera — Questions Modi's ability to connect with youth, reflecting skepticism towards BJP.
Neutral: Bihar Assembly Election 2025: where and how to watch the results live - The Hindu — Provides logistical information without any political stance.
Neutral: Bihar election result 2025: Date,

 ## Project Done by :  

 ##### Dhruv Singla and Maria Thurkadayi